In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import re
import string
import nltk 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import pyarabic.araby as araby
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc
import joblib
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
from torchmetrics.functional import f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from transformers import Trainer, TrainingArguments
import random



In [ ]:
pip install pytorch_pretrained_bert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 817.5 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pad_sequences

  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10217 sha256=ec188c81f11e608c7f5d2555dbeb5fb685cbdd53520a7cfddc0b55393492f15c
  Stored in directory: /root/.cache/pip/wheels/3f/80/05/48e28be7b6bde8b3378f747f50bc32a87cb281c2e89ae74500
Successfully built pad_sequences
Note: you may need to restart the kernel to use updated packages.


In [ ]:
Nile = pd.read_csv("../input/nileuniversity-emotionalcsv/NileUniversity_Emotional.csv")

In [ ]:
Nile

,ID,text,label
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب xD,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none
...,...,...,...
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy


In [ ]:
Nile.head()

,ID,text,label
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب xD,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none


In [ ]:
Nile.tail()

,ID,text,label
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy
10064,10065,AlHamad يطلع ننهم كل شي سيء ووضيع كل خساسه الع...,anger


In [ ]:
Nile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10065 entries, 0 to 10064
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10065 non-null  int64 
 1   text    10064 non-null  object
 2   label   10065 non-null  object
dtypes: int64(1), object(2)
memory usage: 236.0+ KB


Check Nulls

In [ ]:
Nile[Nile.text.isnull()]

,ID,text,label
8619,8620,NaN,love


In [ ]:
Nile.dropna(inplace=True)

In [ ]:
Nile.isnull().sum()

ID       0
text     0
label    0
dtype: int64

Remove stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Replace emoticon with emoji
emoji = {
        ":)" : "🙂",
        ":(" : "🙁",
        "xD" : "😆",
        ":=(": "😭",
        ":'(": "😢",
        ":'‑(": "😢",
        "XD" : "😂",
        ":D" : "🙂",
        "♬" : "موسيقي",
        "♡" : "❤",
        "☻"  : "🙂",
        }

In [ ]:
def emoticon(text):
    for i in range (len(text)):
        message = text.iloc[i]
        seperarate_word = message.split(' ')
        Nile.text.iloc[i] = ""

        for word in seperarate_word:
            text.iloc[i] += emoji.get(word, word) + " "

In [ ]:
emoticon(Nile.text)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
Nile.text[2]

'كتنا نيله ف حظنا الهباب 😆 '

In [ ]:
import emoji
#to count the number of emojis in the dataset
def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

In [ ]:
Nile['emoji_count'] = Nile['text'].apply(lambda x: emoji_counter(x))

In [ ]:
Nile

,ID,text,label,emoji_count
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none,0
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger,0
2,3,كتنا نيله ف حظنا الهباب 😆,sadness,1
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy,0
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none,0
...,...,...,...,...
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness,0
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise,3
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none,1
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy,0


In [ ]:
emojis_data = Nile[Nile['emoji_count'] > 0]

In [ ]:
emojis_data.reset_index(drop=True,inplace=True)

In [ ]:
emojis_data

,ID,text,label,emoji_count
0,3,كتنا نيله ف حظنا الهباب 😆,sadness,1
1,13,يا ويل قلبي منه ي كثر ما اغليه ويا كثر ماله ف...,love,1
2,19,المفروض يلغوها من الاوليمبياد وندخل الاسكواش م...,none,1
3,20,25jan اعوذ بالله مايسوي هذا الرز اللي تحكوا ...,sympathy,1
4,25,بعثه مصر في الاوليمبياد رايحه تتفرج زيها زينا 😂😂,none,2
...,...,...,...,...
2085,10058,للامانه كنت متعاطفه مع هيلاري كثير لانها حرمه ...,sympathy,1
2086,10059,كنت خايفه من هاد اليوم 💔💔💔 فاقولنا يا جدك الج...,fear,3
2087,10060,خايفه نبدا مذاكره ونعمل حاجات مفيده ونسيب الحا...,fear,1
2088,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise,3


In [ ]:
emojis_data.text.iloc[0]

'كتنا نيله ف حظنا الهباب 😆 '

In [ ]:
emojis_data = pd.read_csv("../input/emojis-data/emojis_data3.csv")

In [ ]:
emojis_data

,ID,text,label,emoji_count,emoji
0,3,كتنا نيله ف حظنا الهباب 😆,sadness,1,😆
1,13,يا ويل قلبي منه ي كثر ما اغليه ويا كثر ماله ف...,love,1,❤
2,19,المفروض يلغوها من الاوليمبياد وندخل الاسكواش م...,none,1,😂
3,20,25jan اعوذ بالله مايسوي هذا الرز اللي تحكوا ...,sympathy,1,💔
4,25,بعثه مصر في الاوليمبياد رايحه تتفرج زيها زينا 😂😂,none,2,😂 😂
...,...,...,...,...,...
2085,10058,للامانه كنت متعاطفه مع هيلاري كثير لانها حرمه ...,sympathy,1,😁
2086,10059,كنت خايفه من هاد اليوم 💔💔💔 فاقولنا يا جدك الج...,fear,3,💔 💔 💔
2087,10060,خايفه نبدا مذاكره ونعمل حاجات مفيده ونسيب الحا...,fear,1,😂
2088,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise,3,😲 😲 ❤


In [ ]:
type(emojis_data.emoji.iloc[0])

str

In [ ]:
emojis_data.emoji.value_counts()

🙂        132
💔        111
😂        110
❤️        78
😂 😂       66
        ... 
😍 😂 😂      1
😔 😂        1
😤 😤        1
❤ 🌷        1
😲 😲 ❤      1
Name: emoji, Length: 753, dtype: int64

In [ ]:
Nile.drop("emoji_count", axis=1, inplace=True)

In [ ]:
Nile

,ID,text,label
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب 😆,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none
...,...,...,...
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy


In [ ]:
#Map emojis into words 
emojis = {
        "🙂":"يبتسم",
        "😂":"يضحك",
        "💔":"قلب حزين",
        "🙂":"يبتسم",
        "❤️":"حب",
        "❤️":"حب",
        "😍":"حب",
        "😭":"يبكي",
        "😢":"حزن",
        "😔":"حزن",
        "♥️":"حب",
        "💜":"حب",
        "😅":"يضحك",
        "🙁":"حزين",
        "💕":"حب",
        "💙":"حب",
        "😞":"حزين",
        "😊":"سعادة",
        "👏":"يصفق",
        "👌":"احسنت",
        "😴":"ينام",
        "😀":"يضحك",
        "😌":"حزين",
        "🌹":"وردة",
        "🙈":"حب",
        "😄":"يضحك",
        "😐":"محايد",
        "✌️":"منتصر",
        "✨":"نجمه",
        "🤔":"تفكير",
        "😏":"يستهزء",
        "😒":"يستهزء",
        "🙄":"ملل",
        "😕":"عصبية",
        "😃":"يضحك",
        "🌸":"وردة",
        "😓":"حزن",
        "💞":"حب",
        "💗":"حب",
        "😑":"منزعج",
        "💭":"تفكير",
        "😎":"ثقة",
        "💛":"حب",
        "😩":"حزين",
        "💪":"عضلات",
        "👍":"موافق",
        "🙏🏻":"رجاء طلب",
        "😳":"مصدوم",
        "👏🏼":"تصفيق",
        "🎶":"موسيقي",
        "🌚":"مبتسم",
        "💚":"حب",
        "💘":"حب",
        "🍃":"سلام",
        "☺️":"يضحك",
        "🐸":"ضفدع",
        "😶":"صامت",
        "✌️":"نصر",
        "✋🏻":"توقف",
        "😉":"غمزة",
        "🌷":"وردة",
        "🙃":"مبتسم",
        "😫":"منزعج",
        "😨":"مصدوم",
        "🎼 ":"موسيقي",
        "🍁":"وردة",
        "🍂":"وردة",
        "💟":"حب",
        "😪":"حزن",
        "😆":"يضحك",
        "😣":"استياء",
        "☺️":"حب",
        "😱":"خائف",
        "😁":"يضحك",
        "😖":"استياء",
        "🏃🏼":"يجري",
        "😡":"غضب",
        "🚶":"يسير",
        "🤕":"مرض",
        "‼️":"تعجب",
        "🕊":"طائر",
        "👌🏻":"احسنت",
        "❣️":"حب",
        "🙊":"مصدوم",
        "💃":"سعادة مرح",
        "💃🏼":"سعادة مرح",
        "😜":"مرح",
        "👊":"ضربة",
        "😟":"استياء",
        "💖":"حب",
        "😥":"حزين",
        "🎻":"موسيقي",
        "✒️":"يكتب",
        "🚶🏻":"يسير",
        "💎":"الماظ",
        "😷":"وباء مرض",
        "☝️":"واحد",
        "🚬":"تدخين",
        "💐" : "ورد",
        "🌞" : "شمس",
        "👆" : "لاعلي",
        "⚠️" :"تحذير",
        "🤗" : "اهتمام",
        "✖️": "غلط",
        "📍"  : "مكان",
        "👸" : "ملكه",
        "👑" : "تاج",
        "✔️" : "صح",
        "💌": "قلب",
        "😲" : "متفاجئ",
        "💦": "ماء",
        "🚫" : "خطا",
        "👏🏻" : "عمل رائع",
        "🏊" :"يسبح",
        "👍🏻": "تمام",
        "⭕️" :"صحيح",
        "🎷" : "ساكسفون",
        "👋": "تلويح باليد",
        "✌🏼": "علامه النصر",
        "🌝":"مبتسم",
        "➿"  : "عقده مزدوجه",
        "💪🏼" : "قوي",
        "📩":  "تواصل معي",
        "☕️": "قهوه",
        "😧" : "قلق و صدمة",
        "🗨": "رسالة",   
        "❗️" :"تعجب",
        "🙆🏻": "اشاره موافقه",
        "👯" :"اخوات",
        "©️" :  "رمز",
        "👵🏽" :"سيده عجوزه",
        "🐣": "مرح",  
        "🙌": "تشجيع",
        "🙇": "شخص ينحني",
        "👐🏽":"ايدي مفتوحه",    
        "👌🏽": "احسنت",
        "⁉️" : "استنكار",
        "⚽️": "كوره",
        "🕶" :"حب",
        "🎈" :"بالون",    
        "🎀": "ورده",
        "💵": "فلوس",   
        "😋":  "جائع",
        "😛":  "يغيظ",
        "😠":  "غاضب",
        "✍🏻":  "يكتب",
        "🌾":  "ارز",
        "👣":  "اثر قدمين",
        "❌":"رفض",
        "🍟":"طعام",
        "👬":"صداقة",
        "🐰":"ارنب",
        "☂️":"مطر",
        "⚜️":"مملكة فرنسا",
        "🐑":"خروف",
        "🗣":"تنبيه",
        "👌🏼":"احسنت",
        "☘️":"مرح",
        "😮":"صدمة",
        "😦":"قلق",
        "⭕️":"صحيح",
        "✏️":"قلم",
        "ℹ️":"معلومات",
        "🙍🏻":"بنت",
        "⚪️":"ابيض",
        "🐤":"مرح",
        "💫":"مرح",
        "💝":"حب",
        "🍔":"طعام",
        "❤️":"حب",
        "✈️":"سفر",
        "🏃🏻‍♀️":"يسير",
        "🍳":"ذكر",
        "🎤":"مايك غناء",
        "🎾":"كره",
        "🐔":"دجاجة",
        "🙋":"سؤال",
        "📮":"بحر",
        "💉":"دواء",
        "🙏🏼":"رجاء طلب",
        "💂🏿 ":"حارس",
        "🎬":"سينما",
        "♦️":"مرح",
        "💡":"قكرة",
        "‼️":"تعجب",
        "👼":"طفل",
        "🔑":"مفتاح",
        "♥️":"حب",
        "🕋":"كعبة",
        "🐓":"دجاجة",
        "💩":"مقرف",
        "👽":"فضائي",
        "☔️":"مطر",
        "🍷":"عصير",
        "🌟":"نجمة",
        "☁️":"سحب",
        "👃":"معترض",
        "🌺":"وردة",
        "🔪":"سكينة",
        "♨️":"سخونة",
        "👊🏼":"ضرب",
        "✏️":"قلم",
        "🚶🏾‍♀️":"يسير",
        "👊":"ضربة",
        "◾️":"وقف",
        "😚":"حب",
        "🔸":"مرح",
        "👎🏻":"لا يعجبني",
        "👊🏽":"ضربة",
        "😙":"حب",
        "🎥":"تصوير",
        "👉":"جذب انتباه",
        "👏🏽":"يصفق",
        "💪🏻":"عضلات",
        "🏴":"اسود",
        "🔥":"حريق",  
        "😬":"عدم الراحة",   
        "👊🏿":"يضرب",    
        "🌿":"ورقه شجره",     
        "✋🏼":"قف",    
        "👐":"ايدي مفتوحه",      
        "☠️":"وجه مرعب",     
        "🎉":"يهنئ",      
        "🔕" :"صامت",
        "😿":" حزين",      
        "☹️":"يائس",     
        "😘" :"حب",     
        "😰" :"خوف و حزن",
        "🌼":"ورده",      
        "💋":  "بوسه",
        "👇":"لاسفل",     
        "❣️":"حب",     
        "🎧":"سماعات",
        "📝":"يكتب",      
        "😇":"مبهج",      
        "😈":"رعب",      
        "🏃":"يجري",      
        "✌🏻":"النصر", 
        "👎":"غير موافق",      
        "🔐":"قفل", 
        "🔫":"مرح",      
        "❗️":"تعجب",
        "👈":"لليمين",
        "™️":"رمز",    
        "🚶🏽":"يتمشي",    
        "😯":"متفاجأ",  
        "✊":"يد مغلقه",    
        "😻":"اعجاب",    
        "🙉" :"مرح",    
        "👧":"طفله صغيره", 
        "💤":"ينام", 
        "👀":"ينظر",     
        "✍🏻":"يكتب",  
        "❄️":"تلج",
        "🖋":"قلم",      
        "🎩":"كاب",      
        "☕️":"قهوه",  
        "🔴":"دائره حمراء",   
        "💀":"رعب",
        "😤":"منزعج", 
        "💪🏽":"قوه",     
        "😹":"ضحك",     
        "💓":"حب",      
        "☄️ ":"نار",     
        "👻":"مرح",
        "🙂":"يبتسم",
        "😂":"يضحك",
        "💔":"قلب حزين",
        "🙂":"يبتسم",
        "❤️":"حب",
        "❤️":"حب",
        "😍":"حب",
        "😭":"يبكي",
        "😢":"حزن",
        "😔":"حزن",
        "♥️":"حب",
        "💜":"حب",
        "😅":"يضحك",
        "🙁":"حزين",
        "💕":"حب",
        "💙":"حب",
        "😞":"حزين",
        "😊":"سعادة",
        "👏":"يصفق",
        "👌":"احسنت",
        "😴":"ينام",
        "😀":"يضحك",
        "😌":"حزين",
        "🌹":"وردة",
        "🙈":"حب",
        "😄":"يضحك",
        "😐":"محايد",
        "✌️":"منتصر",
        "✨":"نجمه",
        "🤔":"تفكير",
        "😏":"يستهزء",
        "😒":"يستهزء",
        "🙄":"ملل",
        "😕":"عصبية",
        "😃":"يضحك",
        "🌸":"وردة",
        "😓":"حزن",
        "💞":"حب",
        "💗":"حب",
        "😑":"منزعج",
        "💭":"تفكير",
        "😎":"ثقة",
        "💛":"حب",
        "😩":"حزين",
        "💪":"عضلات",
        "👍":"موافق",
        "🙏🏻":"رجاء طلب",
        "😳":"مصدوم",
        "👏🏼":"تصفيق",
        "🎶":"موسيقي",
        "🌚":"صمت",
        "💚":"حب",
        "🙏":"رجاء طلب",
        "💘":"حب",
        "🍃":"سلام",
        "☺️":"يضحك",
        "🐸":"ضفدع",
        "😶":"مصدوم",
        "✌️":"مرح",
        "✋🏻":"توقف",
        "😉":"غمزة",
        "🌷":"حب",
        "🙃":"مبتسم",
        "😫":"حزين",
        "😨":"مصدوم",
        "🎼 ":"موسيقي",
        "🍁":"مرح",
        "🍂":"مرح",
        "💟":"حب",
        "😪":"حزن",
        "😆":"يضحك",
        "😣":"استياء",
        "☺️":"حب",
        "😱":"كارثة",
        "😁":"يضحك",
        "😖":"استياء",
        "🏃🏼":"يجري",
        "😡":"غضب",
        "🚶":"يسير",
        "🤕":"مرض",
        "‼️":"تعجب",
        "🕊":"طائر",
        "👌🏻":"احسنت",
        "❣️":"حب",
        "🙊":"مصدوم",
        "💃":"سعادة مرح",
        "💃🏼":"سعادة مرح",
        "😜":"مرح",
        "👊":"ضربة",
        "😟":"استياء",
        "💖":"حب",
        "😥":"حزن",
        "🎻":"موسيقي",
        "✒️":"يكتب",
        "🚶🏻":"يسير",
        "💎":"الماظ",
        "😷":"وباء مرض",
        "☝️":"واحد",
        "🚬":"تدخين",
        "💐" : "ورد",
        "🌞" : "شمس",
        "👆" : "الاول",
        "⚠️" :"تحذير",
        "🤗" : "احتواء",
        "✖️": "غلط",
        "📍"  : "مكان",
        "👸" : "ملكه",
        "👑" : "تاج",
        "✔️" : "صح",
        "💌": "قلب",
        "😲" : "مندهش",
        "💦": "ماء",
        "🚫" : "خطا",
        "👏🏻" : "برافو",
        "🏊" :"يسبح",
        "👍🏻": "تمام",
        "⭕️" :"دائره كبيره",
        "🎷" : "ساكسفون",
        "👋": "تلويح باليد",
        "✌🏼": "علامه النصر",
        "🌝":"مبتسم",
        "➿"  : "عقده مزدوجه",
        "💪🏼" : "قوي",
        "📩":  "تواصل معي",
        "☕️": "قهوه",
        "😧" : "قلق و صدمة",
        "🗨": "رسالة",   
        "❗️" :"تعجب",
        "🙆🏻": "اشاره موافقه",
        "👯" :"اخوات",
        "©" :  "رمز",
        "👵🏽" :"سيده عجوزه",
        "🐣": "كتكوت",  
        "🙌": "تشجيع",
        "🙇": "شخص ينحني",
        "👐🏽":"ايدي مفتوحه",    
        "👌🏽": "بالظبط",
        "⁉️" : "استنكار",
        "⚽️": "كوره",
        "🕶" :"حب",
        "🎈" :"بالون",    
        "🎀":    "ورده",
        "💵":  "فلوس",   
        "😋":  "جائع",
        "😛":  "يغيظ",
        "😠":  "غاضب",
        "✍🏻":  "يكتب",
        "🌾":  "ارز",
        "👣":  "اثر قدمين",
        "❌":"رفض",
        "🍟":"طعام",
        "👬":"صداقة",
        "🐰":"ارنب",
        "☂️":"مطر",
        "⚜️":"مملكة فرنسا",
        "🐑":"خروف",
        "🗣":"صوت مرتفع",
        "👌🏼":"احسنت",
        "☘️":"مرح",
        "😮":"صدمة",
        "😦":"قلق",
        "⭕️":"الحق",
        "✏️":"قلم",
        "ℹ️":"معلومات",
        "🙍🏻":"رفض",
        "⚪️":"نضارة نقاء",
        "🐤":"حزن",
        "💫":"مرح",
        "💝":"حب",
        "🍔":"طعام",
        "❤️":"حب",
        "✈️":"سفر",
        "🏃🏻‍♀️":"يسير",
        "🍳":"ذكر",
        "🎤":"مايك غناء",
        "🎾":"كره",
        "🐔":"دجاجة",
        "🙋":"سؤال",
        "📮":"بحر",
        "💉":"دواء",
        "🙏🏼":"رجاء طلب",
        "💂🏿 ":"حارس",
        "🎬":"سينما",
        "♦️":"مرح",
        "💡":"قكرة",
        "‼️":"تعجب",
        "👼":"طفل",
        "🔑":"مفتاح",
        "♥️":"حب",
        "🕋":"كعبة",
        "🐓":"دجاجة",
        "💩":"معترض",
        "👽":"فضائي",
        "☔️":"مطر",
        "🍷":"عصير",
        "🌟":"نجمة",
        "☁️":"سحب",
        "👃":"معترض",
        "🌺":"مرح",
        "🔪":"سكينة",
        "♨️":"سخونية",
        "👊🏼":"ضرب",
        "✏️":"قلم",
        "🚶🏾‍♀️":"يسير",
        "👊":"ضربة",
        "◾️":"وقف",
        "😚":"حب",
        "🔸":"مرح",
        "👎🏻":"لا يعجبني",
        "👊🏽":"ضربة",
        "😙":"حب",
        "🎥":"تصوير",
        "👉":"جذب انتباه",
        "👏🏽":"يصفق",
        "💪🏻":"عضلات",
        "🏴":"اسود",
        "🔥":"حريق",  
        "😬":"عدم الراحة",   
        "👊🏿":"يضرب",    
        "🌿":"ورقه شجره",     
        "✋🏼":"كف ايد",    
        "👐":"ايدي مفتوحه",      
        "☠️":"وجه مرعب",     
        "🎉":"يهنئ",      
        "🔕" :"صامت",
        "😿":"وجه حزين",      
        "☹️":"وجه يائس",     
        "😘" :"حب",     
        "😰" :"خوف و حزن",
        "🌼":"ورده",      
        "💋":  "بوسه",
        "👇":"لاسفل",     
        "❣️":"حب",     
        "🎧":"سماعات",
        "📝":"يكتب",      
        "😇":"دايخ",      
        "😈":"رعب",      
        "🏃":"يجري",      
        "✌🏻":"علامه النصر",    
        "🔫":"يضرب",      
        "❗️":"تعجب",
        "👎":"غير موافق",      
        "🔐":"قفل",      
        "👈":"لليمين",
        "™":"رمز",    
        "🚶🏽":"يتمشي",    
        "😯":"متفاجأ",  
        "✊":"يد مغلقه",    
        "😻":"اعجاب",    
        "🙉" :"قرد",    
        "👧":"طفله صغيره",     
        "🔴":"دائره حمراء",      
        "💪🏽":"قوه",     
        "💤":"ينام",     
        "👀":"ينظر",     
        "✍🏻":"يكتب",  
        "❄️":"تلج",
        "💀":"رعب",   
        "😤":"وجه عابس",      
        "🖋":"قلم",      
        "🎩":"كاب",      
        "☕️":"قهوه",     
        "😹":"ضحك",     
        "💓":"حب",      
        "☄️ ":"نار",     
        "👻":"رعب",
        }
        


In [ ]:
def emoticons(text):
    for i in range (len(text)):
        message = text.iloc[i]
        seperarate_word = message.split(' ')
        Nile.text.iloc[i] = ""

        for word in seperarate_word:
            text.iloc[i] += emojis.get(word, word) + " "

In [ ]:
emoticons(Nile.text)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
Nile.text[2]

'كتنا نيله ف حظنا الهباب يضحك  '

In [ ]:
#Remove stopwords 
stopwords_list = stopwords.words('arabic')
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
listToStr

'إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هن

In [ ]:
print(len(stopwords_list))

754


Clean text

In [ ]:
for letter in '#.][!XR':
    Nile['text'] = Nile['text'].astype(str).str.replace(letter,'')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [ ]:
#Remove punctaution
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
#Normalize
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [ ]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def remove(string):
    return " ".join(string.split())

Remove prefixes and suffixes

In [ ]:
st = ISRIStemmer()
def roots(text):
    return st.stem(text)

In [ ]:
def pre(text):
    return st.pre32(text)

In [ ]:
def suf(text):
    return st.suf32(text)

In [ ]:
def processPost(text): 
    # remove Negation
    text=text.replace("ﷺ", "")
    text=text.replace("لا", "")
    text=text.replace("ما", "")
    text=text.replace("لم", "")
    text=text.replace("بدون", "")
    text=text.replace("من دون", "")
    text=text.replace("لن", "")
    text=text.replace("لات", "")
   
    #Replace @username with empty string
    text = re.sub('@[^\s]+', ' ', text)
    # remove english letters
    text = re.sub('[a-zA-Z0-9_]|#|http\S+', '', text)
    
    #remove links
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)

    # remove punctuations
    text = remove_punctuations(text)
    
    # normalize the tweet
    text = normalize_arabic(text)
    # remove Diacritics
    text = araby.strip_diacritics(text)
    
    # remove repeated letters
    text = remove_repeating_char(text)
    # remove space
    text = remove(text)
    text = ' '.join(word for word in text.split(' ') if word not in stopwords_list)
    #remove uni code 
    text = re.sub(r'\W+', ' ', text)
    # remove suffixes
    text= suf(text)
    # remove prefixes
    text= pre(text)
    
    return text

In [ ]:
Nile["text"] = Nile['text'].apply(lambda x: processPost(x))

In [ ]:
Nile['text']

0                         اوليمبياد الجايه هكون لسه الكليه
1        عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...
2                               كتنا نيله حظنا الهباب يضحك
3          جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي
4        اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...
                               ...                        
10060    جعه حفله عمرو دياب خلصت نريح شويه ونبدا نكتئب ...
10061                                        ايه دا اوزيل 
10062     عملتلها ريتويت بمناسبه ساره بتاعه اوليمبياد يضحك
10063            وعليك قبا يانجم اجوم ياعندليب الحب واحساس
10064    يطلع نهم شي سيء وضيع خساسه العا تجمعت ايرانين ...
Name: text, Length: 10064, dtype: object

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
Nile["text"] = Nile["text"].apply(tokenizer.tokenize)

In [ ]:
Nile['text'] = Nile['text'].apply(lambda x: ' '.join([word for word in x ]))
Nile["text"]

0                         اوليمبياد الجايه هكون لسه الكليه
1        عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...
2                               كتنا نيله حظنا الهباب يضحك
3          جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي
4        اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...
                               ...                        
10060    جعه حفله عمرو دياب خلصت نريح شويه ونبدا نكتئب ...
10061                                         ايه دا اوزيل
10062     عملتلها ريتويت بمناسبه ساره بتاعه اوليمبياد يضحك
10063            وعليك قبا يانجم اجوم ياعندليب الحب واحساس
10064    يطلع نهم شي سيء وضيع خساسه العا تجمعت ايرانين ...
Name: text, Length: 10064, dtype: object

Save cleaning_data

In [ ]:
Nile.to_csv('clean_data.csv',index=False)

Reducing words to their roots

In [ ]:
Nile['text'] = Nile['text']. apply(lambda x: [st.stem(y) for y in x.split()])
Nile['text'] = Nile['text'].apply(lambda x: ' '.join([word for word in x ]))
Nile['text']

0                                اوليمبياد جيه هكن لسه كله
1        عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...
2                                      كتن نيل حظن هبب يضح
3                          جمع نرد حقق هدف ونس تلق حرس ارم
4        اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...
                               ...                        
10060    جعه حفل عمرو ديب خلص نرح شوه نبد كتئب عشم نفس ...
10061                                           ايه دا وزل
10062                 عملتل ريتو نسب سره بتع اوليمبياد يضح
10063                     وعل قبا نجم اجم ياعندليب لحب حسس
10064    طلع نهم شي سيء وضع خسس الع جمع يرن الي لفت بحر...
Name: text, Length: 10064, dtype: object

In [ ]:
Nile.to_csv('clean_data_2.csv',index=False)

data with_out ISRIStemmer

In [ ]:
path_input = './clean_data.csv'
df = pd.read_csv(path_input)
df.head()

,ID,text,label
0,1,اوليمبياد الجايه هكون لسه الكليه,none
1,2,عجز اوازنه وصل اتج احلي يعني لسه اقل نفلس بهاي...,anger
2,3,كتنا نيله حظنا الهباب يضحك,sadness
3,4,جميعنا نريد تحقيق اهدافنا تونس تالقت حراسه ارمي,joy
4,5,اوليمبياد نظامها مختلف ومواعيد اونديال مكانتش ...,none


In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

ID       0
text     0
label    0
dtype: int64

data with ISRIStemmer

In [ ]:
path_input = './clean_data_2.csv'
dff = pd.read_csv(path_input)
dff.head()

,ID,text,label
0,1,اوليمبياد جيه هكن لسه كله,none
1,2,عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...,anger
2,3,كتن نيل حظن هبب يضح,sadness
3,4,جمع نرد حقق هدف ونس تلق حرس ارم,joy
4,5,اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...,none


In [ ]:
dff.dropna(inplace=True)
dff.isnull().sum()

ID       0
text     0
label    0
dtype: int64

**Model without ISRIStemmer

In [ ]:
train, val = train_test_split(df[['label','text']], test_size=0.1, random_state=333)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [ ]:
train

,label,text
9552,4,فاطمه الشرنوبي تحتل اركز اخير مجموعتها سباق مت...
3262,5,اسم الشهابي بطل مصر وافريقيا الجودو هيعب بكره ...
5267,4,دا احنا ناخد اوليمبياد دا بقا اليد
6570,4,ايه كوبايه اوليمبياد دي
378,3,عظيم الحب ان تري نفسك جسد اخر
...,...,...
7539,4,ميداليه دهب اعظم تاريخ اوليمبياد اعظم رياضي ال...
6072,6,يمكن ان تكون داخلي اكثر م
9644,3,تجعل كبريائك يقف طريق عودتك يحمل اكواما الحب
973,0,غالي متطلعش دين ابو


In [ ]:
val

,label,text
7313,7,ه يكون معكم ينصر
5200,7,وحدكم فاله معكم وقلوبنا ايضا فالفرج قريب
5330,6,كانوا جايبين يحل اوليمبياد عايل سبورت قالهم وا...
9071,4,كوره نسائيه اوليمبياد علي ابوظبي سبورت
3116,0,انا ضد ان اي حد يعب اسرائيلي اوليمبياد نه بيعت...
...,...,...
8930,7,ابي يارب انك قبضت روحا غاليه علينا ونحن نملك ا...
4395,4,بنت بلد اسمها كوسوفو اول مره تشارك اوليمبياد ت...
5943,0,ليش تفكر السعوديه زيك حر ايران منبطح حذاء اير
5205,0,مرتضي منصور ده هيودينا دهيه واله هيودينا ايه و...


In [ ]:
#Handling lables
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [ ]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Mini"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "Base": "aubmindlab/bert-base-arabertv02",
                 "MAR" : "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [ ]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [ ]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Mini"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                 "MAR" : ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"train_f1": f1_Score, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"val_f1": f1_Score, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

**Arabert Twitter model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Twitter"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [09:52<00:00, 53.85s/it]

100%|██████████| 11/11 [00:01<00:00,  5.57it/s]

              precision    recall  f1-score   support

       anger       0.70      0.74      0.72       135
        fear       0.96      0.94      0.95       125
         joy       0.69      0.61      0.65       137
        love       0.76      0.80      0.78       117
        none       0.71      0.83      0.76       166
     sadness       0.63      0.59      0.61       122
    surprise       0.59      0.52      0.55        93
    sympathy       0.92      0.88      0.90       111

    accuracy                           0.75      1006
   macro avg       0.75      0.74      0.74      1006
weighted avg       0.75      0.75      0.74      1006



**Arabert Base Model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Base"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

...

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [04:54<00:00, 26.77s/it]

100%|██████████| 11/11 [00:01<00:00,  5.58it/s]

              precision    recall  f1-score   support

       anger       0.69      0.72      0.71       135
        fear       0.96      0.95      0.96       125
         joy       0.64      0.65      0.64       137
        love       0.82      0.74      0.78       117
        none       0.74      0.69      0.71       166
     sadness       0.56      0.64      0.60       122
    surprise       0.51      0.55      0.53        93
    sympathy       0.94      0.88      0.91       111

    accuracy                           0.73      1006
   macro avg       0.73      0.73      0.73      1006
weighted avg       0.74      0.73      0.73      1006



**MARBERT

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MAR"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:01<00:00,  5.56it/s]

              precision    recall  f1-score   support

       anger       0.68      0.64      0.66       135
        fear       0.95      0.94      0.94       125
         joy       0.75      0.55      0.64       137
        love       0.78      0.81      0.79       117
        none       0.70      0.73      0.72       166
     sadness       0.60      0.63      0.62       122
    surprise       0.46      0.60      0.52        93
    sympathy       0.93      0.89      0.91       111

    accuracy                           0.72      1006
   macro avg       0.73      0.73      0.72      1006
weighted avg       0.73      0.72      0.73      1006



**with ISRIStemmer

In [ ]:
Nile['text']

0                                اوليمبياد جيه هكن لسه كله
1        عجز وزن وصل اتج احل يعن لسه اقل فلس هيم لسه تب...
2                                      كتن نيل حظن هبب يضح
3                          جمع نرد حقق هدف ونس تلق حرس ارم
4        اوليمبياد نظم خلف مواعيد ديل مكانتش قرف حجه كا...
                               ...                        
10060    جعه حفل عمرو ديب خلص نرح شوه نبد كتئب عشم نفس ...
10061                                           ايه دا وزل
10062                 عملتل ريتو نسب سره بتع اوليمبياد يضح
10063                     وعل قبا نجم اجم ياعندليب لحب حسس
10064    طلع نهم شي سيء وضع خسس الع جمع يرن الي لفت بحر...
Name: text, Length: 10064, dtype: object

In [ ]:
train, val = train_test_split(dff[['label','text']], test_size=0.1, random_state=333)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [ ]:
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [ ]:
train

,label,text
9552,4,فطم شرنوب حتل ركز اخر جمع سبق متر عدو وبالتالي...
3262,5,اسم شهب بطل مصر فرق جودو هيعب بكر كين صهو اولي...
5267,4,دا احن نخد اوليمبياد دا بقا الد
6570,4,ايه كوب اوليمبياد دي
378,3,عظم لحب ان تري نفس جسد اخر
...,...,...
7539,4,يدل دهب عظم ارخ اوليمبياد عظم ريض ارخ خرج تنس
6072,6,يمكن ان تكون دخل كثر م
9644,3,جعل كبريائ يقف طرق عود حمل اكا لحب
973,0,غلي طلعش دين ابو


In [ ]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Mini"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "Base": "aubmindlab/bert-base-arabertv02",
                 "MAR": "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [ ]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [ ]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Mini"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                 "MAR": ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"train_f1": f1_Score, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_Score = f1_score(out, y, num_classes=n_classes, average='macro')
        metrics = {"val_f1": f1_Score, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

Arabert twitter model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Twitter"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [04:30<00:00, 24.60s/it]

100%|██████████| 11/11 [00:01<00:00,  5.55it/s]

              precision    recall  f1-score   support

       anger       0.61      0.64      0.62       135
        fear       0.95      0.94      0.95       125
         joy       0.62      0.47      0.54       137
        love       0.75      0.70      0.72       117
        none       0.69      0.81      0.74       166
     sadness       0.51      0.64      0.57       122
    surprise       0.49      0.40      0.44        93
    sympathy       0.91      0.83      0.87       111

    accuracy                           0.69      1006
   macro avg       0.69      0.68      0.68      1006
weighted avg       0.69      0.69      0.69      1006



**Arabert Base model

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "Base"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

100%|██████████| 11/11 [00:01<00:00,  5.56it/s]

              precision    recall  f1-score   support

       anger       0.56      0.64      0.60       135
        fear       0.97      0.94      0.95       125
         joy       0.68      0.46      0.55       137
        love       0.69      0.74      0.71       117
        none       0.64      0.71      0.67       166
     sadness       0.55      0.43      0.48       122
    surprise       0.42      0.53      0.46        93
    sympathy       0.85      0.84      0.84       111

    accuracy                           0.66      1006
   macro avg       0.67      0.66      0.66      1006
weighted avg       0.67      0.66      0.66      1006



**MARBERT

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MAR"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                batch_size=100, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_f1")]) #callbacks=[EarlyStopping(monitor="val_f1")]
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))


100%|██████████| 11/11 [04:42<00:00, 25.69s/it]

100%|██████████| 11/11 [00:01<00:00,  5.58it/s]

              precision    recall  f1-score   support

       anger       0.47      0.83      0.60       135
        fear       0.98      0.93      0.95       125
         joy       0.49      0.55      0.52       137
        love       0.82      0.70      0.76       117
        none       0.69      0.40      0.50       166
     sadness       0.49      0.32      0.39       122
    surprise       0.39      0.55      0.46        93
    sympathy       0.93      0.73      0.82       111

    accuracy                           0.62      1006
   macro avg       0.66      0.63      0.62      1006
weighted avg       0.66      0.62      0.62      1006



Machine Learning


TFIDF

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

unigramdataGet= word_vectorizer.fit_transform(dff['text'].astype('str'))
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0في,0وك,ıs,ɺi,ρɺɹ,алепо,детеи,из,мстит,от,...,ڪم,ڪيب,ګبر,ګثر,ھم,ۆايام,ۆرق,ۆفا,یفق,۶ظم
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pro= preprocessing.LabelEncoder()
encpro=pro.fit_transform(dff['label'])
dff['label'] = encpro
dff['label']

0        4
1        0
2        5
3        2
4        4
        ..
10059    5
10060    6
10061    4
10062    2
10063    0
Name: label, Length: 10055, dtype: int64

In [ ]:
y=dff['label']
X=unigramdata_features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=333)

LogisticRegression

In [ ]:
LR= LogisticRegression(penalty = 'l2', C = 1)
LR= LR.fit(X_train , y_train)
y_pred = LR.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[[175   0  20   6  15  25  25   9]
 [  2 219   1   2   4   4   3   0]
 [ 16   0 137  31  44  17  20   3]
 [  8   1  33 175   0  17   4   5]
 [  7   1  22   1 262  24   7   2]
 [ 31   2  22  19  35 113  16   7]
 [ 26   1  19   4  56  18  86   1]
 [  2   1   7   9   0  10   3 176]]


              precision    recall  f1-score   support

           0       0.66      0.64      0.65       275
           1       0.97      0.93      0.95       235
           2       0.52      0.51      0.52       268
           3       0.71      0.72      0.71       243
           4       0.63      0.80      0.71       326
           5       0.50      0.46      0.48       245
           6       0.52      0.41      0.46       211
           7       0.87      0.85      0.86       208

    accuracy                           0.67      2011
   macro avg       0.67      0.66      0.67      2011
weighted avg       0.67      0.67      0.66      2011



SVM

In [ ]:
from sklearn.svm import SVC
clf=SVC(kernel='linear')
clf.fit(X_train , y_train)
y_pred =clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[162   3  21   9  14  32  25   9]
 [  2 217   1   1   4   7   1   2]
 [ 17   1 145  28  43  11  18   5]
 [  9   2  42 168   0  16   3   3]
 [ 14   1  29   1 253  19   8   1]
 [ 41   4  27  15  33 104  15   6]
 [ 29   1  19   3  52  17  87   3]
 [  5   0   7   9   0   9   3 175]]


              precision    recall  f1-score   support

           0       0.58      0.59      0.58       275
           1       0.95      0.92      0.94       235
           2       0.50      0.54      0.52       268
           3       0.72      0.69      0.70       243
           4       0.63      0.78      0.70       326
           5       0.48      0.42      0.45       245
           6       0.54      0.41      0.47       211
           7       0.86      0.84      0.85       208

    accuracy                           0.65      2011
   macro avg       0.66      0.65      0.65      2011
weighted avg       0.65      0.65      0.65      2011



Optimization

In [ ]:
X = np.array(X_train)
Y = np.array( y_train)
# Always scale the input. The most convenient way is to use a pipeline.
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X, Y)


y_pred =clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[[165   0  25   4  15  44  18   4]
 [  4 188   8   5   3  20   6   1]
 [ 10   2 152  20  37  27  18   2]
 [ 11   2  41 153   0  29   6   1]
 [  9   0  44   1 229  27  14   2]
 [ 21   5  31  19  32 118  15   4]
 [ 19   0  34   4  50  24  80   0]
 [  9   1  22  11   0  12   8 145]]


              precision    recall  f1-score   support

           0       0.67      0.60      0.63       275
           1       0.95      0.80      0.87       235
           2       0.43      0.57      0.49       268
           3       0.71      0.63      0.67       243
           4       0.63      0.70      0.66       326
           5       0.39      0.48      0.43       245
           6       0.48      0.38      0.43       211
           7       0.91      0.70      0.79       208

    accuracy                           0.61      2011
   macro avg       0.65      0.61      0.62      2011
weighted avg       0.64      0.61      0.62      2011

